In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
from db.db_operations import execute_db_operations

from generators.full_generators import (
    create_company_data
)

company_name = "Norlys"

In [2]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: Norlys...
No context report found for Norlys. Generating a new one...
Loading and reading year-end report for Norlys.
This usually takes 5-10 mins.
Context data for Norlys generated: {'company_name': 'Norlys', 'count_employee': 580, 'count_department': 28, 'count_customer': 75, 'count_product': 240, 'count_procurement': 190, 'count_service': 180, 'count_account': 70, 'count_vendor': 75, 'estimated_product': 8104000000, 'estimated_service': 2151000000, 'estimated_overhead': 2880000000, 'estimated_revenue': 14289000000}
Generating dimensions for company: Norlys.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Accounts data generated.
✔ Customers data generated.
✔ Departments data generated.
Only received 74 rows, expected 75
✖ Generation failed: Runtime error: 'int' object has no attribute 'insert'

=== Attempt 2 ===
✔ Roles and Names generated.


In [6]:
version_tag = company_name.lower() + dt.now().strftime("%h%m%d%M")
execute_db_operations(version_tag)

[INFO] Inserting 10 rows into dim_department using to_sql...
[INFO] Inserting 30 rows into dim_customer using to_sql...
[INFO] Inserting 100 rows into dim_product using to_sql...
[INFO] Inserting 30 rows into dim_account using to_sql...
[INFO] Inserting 50 rows into dim_procurement using to_sql...
[INFO] Inserting 50 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_line using to_sql...
[INFO] Inserting 30 rows into dim_vendor using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 50 rows into dim_employee using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 50400 rows into fact_payroll using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain 

[INFO] Inserting 6180 rows into fact_general_ledger using to_sql...


In [3]:
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
